# Markov Chain Monte Carlo

Used when you do not have independence in samples. The **markov property** looks at the most previous sample to help inform the next sample!

This is a stochastic process. Other processes like this are q-theory, brownian motion, and poisson process.

Monte carlo uses these simulated R.V.s to approximate integrals, etc. but the R.V. don't need to be independent in order to approximate integrals. MCMC constructs a dependent sequence of RV that can be used to approximate the integrals like the ordinary MC. The advantages of introducing this dependence is that very general "black box" algorithms (and corresponding theory) are available to perform the required simulations. This page will discuss some basics of Markov chains and MCMC but know that there are very important unanswered questions about how and when MCMC works.

## Definition

A markov chain is just a sequence of R.V. $\{ x_1, x_2, ... \}$ with a specific type of dependence structure. 
In particular, a Markov chain satisfies 

$$P(X_{n+1} \in B | X_1, ..., X_{n-1}, X_n) = P(X_{n+1} \in B | X_n)$$

where $X_{n+1}$ is the cuture, $X_1, ..., X_{n-1}$ is the past, and $X_n$ is the present.
Therefore, this property states that the future is only dependent on the present.
This is called the *markov property*.

Independence is a trivial Markov Chain.

From the markov property, we can argue that the probabilistic properties of the chain are completely determined by 

i. initial distribution for $X_0$

ii. the transition distribution, i.e. distribution of $X_{n+1}$ given $X_n$

Note: Assume that the markov chain is homogeneous (aka, the transition distribution does not depend on $n$). 

Example: **simple random walk**
Let $v_1, v_2, ...$ be iid $\sim Unif(-1,1)$

Set $x_0 = 0$ and $X_n = \sum_{i=1}^n U_i = X_{n-1} + U_n$.
The initial distribution is $P(X_0 = 0) = 1$.
The transition distribution is determined by $$x_n = \begin{cases} 
x_{n-1}-1 & prob.= 1/2\\
x_{n-1}+1 & prob.= 1/2\\
\end{cases}$$

While very simple, the random walk is an important example in probability theory, having connections to advanced things like Brownian Motion. In some conditions, random walk becomes brownian motion.

In [2]:
import numpy as np

# Task: predict next number

# Sample set 1: 
X = np.array([0,1,2,3,4,5,6,7,8,9]*100 + [3])

lag=1

total_count_dict = {}
dependent_count_dict = {}
for i in range(len(X)-lag):

    if X[i] not in total_count_dict:
        total_count_dict[X[i]] = 0
    total_count_dict[X[i]] += 1
    
    if X[i] not in dependent_count_dict:
        dependent_count_dict[X[i]] = {}
    if X[i+lag] not in dependent_count_dict[X[i]]:
        dependent_count_dict[X[i]][X[i+lag]] = 0
    dependent_count_dict[X[i]][X[i+lag]] += 1

# Normalize each value in dependent_count_dict by total_count_dict
for k,d in dependent_count_dict.items():
    for key in d.keys():
        d[key] /= total_count_dict[k]

print("Transition Matrix:")
dependent_count_dict

Transition Matrix:


{0: {1: 1.0},
 1: {2: 1.0},
 2: {3: 1.0},
 3: {4: 1.0},
 4: {5: 1.0},
 5: {6: 1.0},
 6: {7: 1.0},
 7: {8: 1.0},
 8: {9: 1.0},
 9: {0: 0.99, 3: 0.01}}

## Brownian Motion

Multiple plays - i.e. Gambler's ruin problem.

## Discrete time Markov Chain (DTMC)

$P(X_{n+1}=j | X_n = i, X_{n-1}, ..., X_1, X_0) = P(X_{n+1}=j|X_n=i)$

(Markov property for DTMC)

If we have a homogeneous MC, then $P(X_{n+1}=j | X_n =i) = p_{ij} = P(X_1 = j|X_0 = i) = P(X_2=j | X_1 = i) = P(X_3=j|X_2=i) = ...$

We can put the $P_{ij}$ in a matrix over the state space $S = \{ 1, 2, ..., m \}$ into a transition matrix: 

$$
P = \begin{bmatrix}
p_{11} & p_{12} & ... & p_{1m}\\
p_{21} & p_{22} & ... & p_{2m}\\
\vdots &&&\\
p_{m1} & p_{m2} & ... & p_{mm}\\
\end{bmatrix}
$$

To describe completely a DTMC, we need $X_0$ (the initial state) and $P$ (transition probabilities).

Example: Two-state DTMC

$S = \{ 1, 2 \}$  and $P = \begin{bmatrix} \alpha&1-\alpha\\ 1-\beta&\beta\\ \end{bmatrix}$

where $0 < \alpha$ and $\beta < 1$

$p_{11} = \alpha = P(X_1=1 | X_0 = 1)$

The probability $P(X_1 = j | X_0 = i) = P_{ij} = P(X_2 = j | X_1 = i) = P(X_3 = j | X_2 = i) = ... = P(X_{n+1} = j | X_n = i)$ due to time-homogeneous property (if applies). Also, this $P_{ij}$ is a 1-step transition probability. 

$P(X_2 = j | X_0 = i) = P_{ij}^{(2)}$ is a 2-step transition probability.

$P(X_n = j | X_0 = i) = P_{ij}^{(n)}$ is an n-step transition probability.

$$
P^{(n)} = \begin{bmatrix}
p_{11}^{(n)} & p_{12}^{(n)} & ... & p_{1m}^{(n)}\\
\vdots & & & \\
p_{m1}^{(n)} & p_{m2}^{(n)} & ... & p_{mm}^{(n)}\\
\end{bmatrix}
$$

is the n-step transition probability matrix.

**Question:** What is $\lim_{n\xrightarrow{} \infty} P^{(n)}$ (limiting distribution)?

For $i, j$, what is $\lim_{n \xrightarrow{} \infty} p_{ij}^{(n)}$, $\forall i,j$?

What is $\lim_{n \xrightarrow{} \infty} P(X_n = j | X_0 = i)$?

Under some conditions, the DTMC $\{ X_n, n\geq 0 \}$ has a stationary distribution $\Pi$. For the 2-state DTMC example, 

$$\Pi = (\pi_1, \pi_2) = ( P(X_n=1), P(X_n=2) )$$

$P(X_n = j), \forall j \in S$. So, it will converge to some PMF.

Moreover, under some conditions, the stationary distribution $\Pi$ can also be the limiting distribution. We can use the same idea to construct a stationary distribution $f$ (in continuous case). $X_0$ and kernel transition.

In statistical computing, this kernel transition is an algorithm.


### Properties

A state $i$ is _recurrent_ if a chain starting in $i$ will eventually return to $i$ with probability 1. If probability less than one, then it is _transient_. Also, the state is _positive recurrent_ if the expected time to return is finite.

A markov chain is _irreducible_ if there is a positive probability that a chain starting in a state $i$ can reach any state $j$.

A markov chain is _aperiodic_ if, for a starting state $i$, there is no constraint on the time at which the chain can return to state $i$. 

An irreducible, aperiodic Markov chain with all states being positive recurrent is called _ergodic_. 


## Limit Theory

$f$ is a stationary distribution if $X_0 \sim f$ up to $X_n \sim f$ for all $n$. An irreducible and positive recurrent markov chain has at most one stationary distribution. Furthermore, if the chain is ergodic, $\lim_{n\xrightarrow{} \infty} P(X_{m+n} \in B | X_m \in A) = \int_B f(x) dx$ for all $A$, $B$, $m$. Even further, if $\phi(x)$ is integrable, then $\frac{1}{n} \sum_{t=1}^n \phi(X_t) \xrightarrow{} \int \phi(x) f(x) dx$ with probability 1. This is a version of the famous ergodic theorem. It is effectively the law of large numbers for the markov chain.

## Motivation: Why MCMC?

In monte carlo applications, we want to generate random variables with distribution $f$. This could be difficult or impossible to do exactly (for example, if have dependent RVs).

The markov chain monte carlo (MCMC) is designed to construct an ergodic monte carlo with $f$ as its stationary distribution. 

Asymptotically, the chain will resemble samples from $f$. In particular, by the ergodic theorem, expectations wrt $f$ can be approximated by averages. Somewhat surprising is that it is quite easy to construct and simulate a suitable MC, explaining why MCMC methods have become so popular. But, of course, there are practical and theoretical challenges. 

For instance, how do you choose a starting value? Also, the _. Also, when stop (i.e. when converge)?

## Metropolis-Hastings Algorithm (MH)

Let $f(x)$ denote the target distribution pdf. Let $q(x|y)$ denote a conditional pdf for $x$ given $Y=y$ (proposal distribution). This (proposal) pdf should be easy to sample from.

Given $X_0$, the MH algorithm produces a sequence of RVs as follows:

1. Sample $X^\star_t \sim q(X|X_{t-1})$

2. Compute $R = \min \{ 1, \frac{f(x^\star_t) q(x_{t0-} | x^\star_t)}{f(x_{t-1})q(x_t^\star | x_{t-1})} \}$

3. Set $X_t = X_t^\star$ with probability $R$; otherwise, $X_t = X_{t-1}$.

As a note: This is different from accept-reject sampling because there is always a sample which is selected in each iteration (as one difference).

The proposal distribution is not easy to choose, and the performance of the algorithm depends on this choice. Two general strategies are:

**Strategies for selecting a proposal distribution for MH:**

Strategy 1: Take the proposal $q(x|y) = q(x)$, i.e. at each stage of the MH algorithm, $x_t^\star$ does not depend on $x_{t-1}$.

Strategy 2: Take $q(x|y) = q_0(x-y)$ for a symmetric distribution with pdf $q_0$, which amounts to a random walk proposal.

This is one aspect of the MCMC implementation that requires a lot of care from the user. Deeper understanding is needed to really see how the proposal affects the performance. Assuming the proposal is not too bad, then a number of things can be shown about the sequence $\{ x_t, t\geq 1 \}$.

i. the chain is ergodic, and 

ii. the target $f$ is the stationary distribution

In summary, we want a sample from $f(x)$. We use MH to get a sample from $f$. Providing the proposal $q$ is not too bad, we can prove that MH renders a sequence which is ergodic, and can say that $f$ is the stationary distribution of the markov chain (meaning the MH is working)!

TODO: prove MH result!

Consequently, the sequence converges to the stationary distribution and for any integrable function $\phi(x)$ we can approximate (using Limit Theory) integrals with sample averages. 

The markov chain gives sample $f$ and the monte carlo step allows you to approximate the integral.

So, provided that we run the simulation "long enough", we should be able to get arbitrarily good approximations.


**Example: ** The Rayleigh density is $$f(x) = \frac{x}{\sigma^2} e^{- \frac{x^2}{2 \sigma^2}}$$ where $x \geq 0$ and $\sigma > 0$

Use the chi-squared distribution with df $x_t$ as the proposal distribution ($q$). The MH algorithm for the Rayleigh distribution is as follows: 

Step 1: Set $q(.|y)$ to the density of $\chi^2_{(y)}$.

Step 2: Generate $X_0$ from distribution $\chi^2_{(1)}$ and store in $X[1]$ (or in python, $X[0]$).

Step 3: Repeat for $i=2, ..., N$: For each, 

Step 3a: Generate $X_t^\star$ from $\chi^2_{(X_t)}$ where the sample is $\chi^2(df=X[i-1])$

Step 3b: Generate $U$ from $unif(0,1)$.

Step 3c: With $X_t = X[i-1]$, compute $r(x^\star_t, x_{t-1}) = \frac{f(x^\star_t) q(x_{t0-} | x^\star_t)}{f(x_{t-1})q(x_t^\star | x_{t-1})}$

If $U \leq r(x^\star_t, x_{t-1})$, set $x_t = x^\star_t$; otherwise, set $x_t = x_{t-1}$.

Step 3d: Increment t

## Gibbs Sampler

Suppose we have a **multivariate target distribution** $f$. MH can be applied to such a problem, but there are challenges in constructing a good proposal over multiple dimensions.

Idea: Sample one dimension at a time!

Question: How to carry out the sampling so that it will approximate the target at least in a limit?

Answer: Gibb's sampler!

Suppose we have a trivariate target $f(x_1, x_2, x_3)$. Suppose we can write down the set of full conditional  $f(x_1|x_2,x_3)$, $f(x_2|x_1,x_3)$, and $f(x_3|x_1,x_2)$ and that these can be sampled from.

Gibbs sampler generates a sequence $\{ x^{(t)}, t\geq 0\}$ by iteratively sampling from the conditionals 

$$x_1^{(t)} \sim f(x_1 | x_2^{(t-1)}, x_3^{(t-1)})$$

$$x_2^{(t)} \sim f(x_2 | x_1^{(t)}, x_3^{(t-1)})$$

$$x_3^{(t)} \sim f(x_3 | x_1^{(t)}, x_2^{(t)})$$

Let's say you have $X_1^{(0)}$, $X_{2}^{(0)}$, $X_3^{(0)}$

$$X_1^{(1)} \sim f(x_1 | x_2^{(0)}, x_3^{(0)})$$

$$X_2^{(1)} \sim f(x_2 | x_1^{(1)}, x_3^{(0)})$$

$$X_3^{(1)} \sim f(x_3 | x_1^{(1)}, x_2^{(1)})$$

**Example:** Bivariate normal

A super simple Gibbs example is the bivariate normal distribution. Suppose $\vec{x} = (x_1, x_2)$ is a bivariate normal with correlation $\rho$.

Full conditionals are easy to write down (for normals)

$E[X_2 | X_1] = \mu_2 + \rho \frac{\sigma_2}{\sigma_1} (x_1 - \mu_1)$

^ Notice, this is linear regression! $x_2 = a x_1 + b$

$Var(X_2 | X_1) = (1 - \rho)^2 \sigma_2^2$

$X_1 | X_2 \sim N(\mu_1 + \rho \frac{\sigma_2}{\sigma_1} (x_2 - \mu_2), (1 - \rho)^2 \sigma_1^2)$

$X_2 | X_1 \sim N(\mu_2 + \rho \frac{\sigma_2}{\sigma_1} (x_1 - \mu_1), (1 - \rho)^2 \sigma_2^2)$

Gibb's sampler

$x_1^{(t)} \sim N(\mu_1 + \rho \frac{\sigma_2}{\sigma_1} (x_2^{(t-1)} - \mu_2), (1 - \rho)^2 \sigma_1^2)$

$x_2^{(t)} \sim N(\mu_2 + \rho \frac{\sigma_2}{\sigma_1} (x_1^{(t)} - \mu_1), (1 - \rho)^2 \sigma_2^2)$

Not as efficient as direct sampling (e.g. accept-reject), but works fine.


Let's say you have a certain target distribution $f$ but you are not able to get a closed form of the condition (as shown above). Here, we can just use monte carlo. 

## Some MCMC diagnostics

